In [2]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA

In [3]:
#read data
data_set_path = "./4 The Local Elections"
x_horsens = np.load("{}/X_Horsens.npy".format(data_set_path))
y_horsens = np.load("{}/Y_Horsens.npy".format(data_set_path))
x_databorg = np.load("{}/X_Databorg.npy".format(data_set_path))
y_databorg = np.load("{}/Y_Databorg.npy".format(data_set_path))

ValueError: Cannot load file containing pickled data when allow_pickle=False

__(a)__ How well do you (intuitively) expect that we can predict the partisan affiliation of a candidate based on their answers to the test?

The result can get clear predict which has very clear boundaries.

__(b)__ Based on the answers from all 83 candidates for the Horsens city council, perform a Principal Component Analysis with 2 principal components. Plot the results in a figure using these 2 components as the axes. Label the points with the party letter and the appropriate color.

In [ ]:
#PCA
pca_horsens = PCA(n_components=2)
x_horsens_transform = pca_horsens.fit_transform(x_horsens)

In [ ]:
#political parties
label_dict = {}
colors_horsens = []
for label in y_horsens:
    if label not in label_dict.keys():
        label_dict[label] = len(label_dict)
    colors_horsens.append(label_dict[label])
colors = np.round(np.array(colors_horsens)/(len(label_dict)-1)*100)
print(label_dict)
plt.scatter(x_horsens_transform[:,0],x_horsens_transform[:,1],c=colors_horsens, cmap='tab20')
plt.title("political parties (horsens)")
plt.show()

__(c)__ Comment on the results. You may consider the following questions for inspiration: Can the political parties be separated? Can the typical distinction of "left-wing" and "right-wing" be discerned? Which of the 18 questions (features) are most important?

In [ ]:
#political position (left:0,right:1)
position_dict = {'A': 0, 'B': 0, 'C': 1, 'D': 1, 'F': 0, 'I': 1, 'O': 1, 'V': 1, 'Z': 0}
colors = []
for label in y_horsens:
    colors.append(position_dict[label])
colors = np.round(np.array(colors)/(len(label_dict)-1)*100)
print(position_dict)
plt.scatter(x_horsens_transform[:,0],x_horsens_transform[:,1],c=colors, cmap='tab20')
plt.title("political position (horsens)")
plt.show()

In [ ]:
#get max difference in mean
importance_list = np.diagonal(pca_horsens.get_covariance())
importance_pca = np.argmax(importance_list)+1
print(importance_list)
print("question {} is the most important one".format(importance_pca))

The political parties can be separated, but not very clear.

The typical distinction of "left-wing" and "right-wing" can be discerned.

__(d)__ Once again, perform a Principal Component Analysis and visualize the results. Compare the results to those of the Horsens data.

In [ ]:
pca_databorg = PCA(n_components=2)
x_databorg_transform = pca_databorg.fit_transform(x_databorg)

label_dict = {}
colors = []
for label in y_databorg:
    if label not in label_dict.keys():
        label_dict[label] = len(label_dict)
    colors.append(label_dict[label])
colors = np.round(np.array(colors)/(len(label_dict)-1)*100)
print(label_dict)
plt.scatter(x_databorg_transform[:,0],x_databorg_transform[:,1],c=colors, cmap='tab20')
plt.title("political parties (databorg)")
plt.show()
plt.scatter(x_horsens_transform[:,0],x_horsens_transform[:,1],c=colors_horsens, cmap='tab20')
plt.title("political parties (horsens)")
plt.show()

The feature area of each partisan is more clear

__(e)__ Split the data into a training and a validation set, with appropriate fractions.

In [ ]:
training_set = []
validation_set = []
training_label = []
validation_label = []
label_dict = {}
for i,label in enumerate(y_databorg):
    if label not in label_dict.keys():
        label_dict[label] = 0
    label_dict[label] += 1
training_num = {}
for label in label_dict.keys():
    training_num[label] = round(label_dict[label]*0.75)
for i,label in enumerate(y_databorg):
    data = x_databorg[i:i+1,:]+1.5
    if training_num[label]>0:
        training_set.append(data)
        training_label.append(np.array([label]))
        training_num[label] -= 1
    else:
        validation_set.append(data)
        validation_label.append(np.array([label]))
training_set = np.concatenate(training_set,axis=0)
validation_set = np.concatenate(validation_set,axis=0)
training_label = np.concatenate(training_label,axis=0)
validation_label = np.concatenate(validation_label,axis=0)
print("training: {}(75%) | validation: {}(25%)".format(len(training_set),len(validation_set)))

__(f)__ Comment on the basic assumption of the Naive Bayes approach. Is this a reasonable assumption for the problem at hand?

The basic assumption of the Naive Bayes approach ask every feature is independent.
It's not a reasonable assumption for the problem, because not each question is independent, some of them have relationship.

__(g)__ Classify the instances of the validation set using a Naive Bayes approach. Comment on the results.

In [ ]:
from sklearn.naive_bayes import CategoricalNB
model_nb = CategoricalNB()
model_nb.fit(training_set,training_label)
validation_output = model_nb.predict(validation_set)
validation_result = np.where(validation_label==validation_output,1,0)
tpr_nb = validation_result.sum()/validation_result.shape[0]
print("[Naive Bayes Categorical] TPR:{:.2%}".format(tpr_nb))

__(h)__ Using default settings of the _k_-NN classifier, classify the instances of the validation set. Comment on the performance.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as knn
model_knn = knn(n_jobs=-1)
model_knn.fit(training_set,training_label)
tpr = model_knn.score(validation_set,validation_label)
print("[KNN Classifier] K:{} TPR:{:.2%}".format(model_knn.n_neighbors,tpr))
print("The performance of KNN can higher than Naive Bayes")

__(i)__ Play around with different values of _k_. Decide on a "good" value of _k_. Comment on the results.

In [ ]:
max_k = 150
tpr_list = []
for i in range(1,max_k+1):
    model_knn.n_neighbors = i
    tpr = model_knn.score(validation_set,validation_label)
    tpr_list.append(tpr)
best_k = np.argmax(tpr_list)+1
tpr_knn = tpr_list[best_k-1]
print("[KNN Classifier] K:{} TPR:{:.2%}".format(best_k,tpr_knn))
plt.plot(range(1,max_k+1),tpr_list)
plt.title("TPR of K")
plt.show()
print("The max TPR of KNN classifier is {:.2%}".format(tpr_knn))

__(j)__ What is the _minimum_ depth of an appropriate decision tree? Why?

There are 18 questions, it means 18 features, so the maximum depth of appropriate decision tree should be 18.

__(k)__ Build a decision tree with at least the depth from above. Play around with the tree depth. Include a figure that shows some relevant measure of the performance as a function of the tree depth. Comment on any issues of over-fitting. Decide on a tree which you will keep for later use. Can you do better than the _k_-NN classifier?

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model_dt = DecisionTreeClassifier()
model_dt.fit(training_set,training_label)
tpr = model_dt.score(validation_set,validation_label)
print("[Decision Tree Classifier] depth:{} TPR:{:.2%}".format(model_dt.get_depth(),tpr))

In [ ]:
max_depth = 18
start_depth = 1
tpr_list = []
training_tpr_list = []
for i in range(start_depth,max_depth+1):
    model_dt = DecisionTreeClassifier(max_depth=i)
    model_dt.fit(training_set,training_label)
    tpr = model_dt.score(validation_set,validation_label)
    tpr_list.append(tpr)
    training_tpr = model_dt.score(training_set,training_label)
    training_tpr_list.append(training_tpr)
best_depth = np.argmax(tpr_list)+start_depth
tpr_dt = tpr_list[best_depth-start_depth]
print("[Decision Tree Classifier] best_depth:{} TPR:{:.2%}".format(best_depth,tpr_dt))
plt.plot(range(start_depth,max_depth+1),tpr_list)
plt.title("TPR of depth")
plt.show()
print("The performance of Decision Tree is lower than KNN")

In [ ]:
#over-fitting
plt.plot(range(start_depth,max_depth+1),training_tpr_list)
plt.plot(range(start_depth,max_depth+1),tpr_list)
plt.title("training/validation TPR of depth")
plt.show()

The TPR on training set is rising with growing depth, but on the validation set, the TPR growth up first and then drop, that means the model is over-fitting. We can use pre-pruning or post-pruning to avoid the over-fitting. Use a suitable depth to stop build the tree early is a type of pre-pruning, and the post-pruning need build whole decision tree first:

In [ ]:
#pruning decision tree
model_dt = DecisionTreeClassifier()
print("[Decision Tree Classifier] TPR:{:.2%} depth:{}".format(tpr_dt,best_depth))
ccp_alphas = model_dt.cost_complexity_pruning_path(training_set,training_label)["ccp_alphas"]
ccp_alphas = np.unique(ccp_alphas)
tpr_list = []
training_tpr_list = []
for ccp_alpha in ccp_alphas:
    model_dt = DecisionTreeClassifier(ccp_alpha=ccp_alpha)
    model_dt.fit(training_set,training_label)
    tpr = model_dt.score(validation_set,validation_label)
    tpr_list.append(tpr)
    training_tpr = model_dt.score(training_set,training_label)
    training_tpr_list.append(training_tpr)
best_ccp_alpha = ccp_alphas[np.argmax(tpr_list)]
tpr_dt_pruning = np.max(tpr_list)
model_dt = DecisionTreeClassifier(ccp_alpha=best_ccp_alpha)
model_dt.fit(training_set,training_label)
print("[Decision Tree Classifier] TPR:{:.2%} depth:{} best_ccp_alpha:{}".format(tpr_dt_pruning,model_dt.get_depth(),best_ccp_alpha))
plt.plot(ccp_alphas,training_tpr_list)
plt.plot(ccp_alphas,tpr_list)
plt.title("TPR of ccp_alpha")
plt.show()

Pre-pruning use very less time, but it will lead to under-fitting. The post-pruning will not lead to under-fitting easily, but it will cost more time.

__(l)__ What are the most important features? Visualize this in an appropriate way. Does it match what you would expect? Compare to the results of the PCA analysis. Do we expect them to be the same? Why/why not?

In [ ]:
importance_list = model_dt.feature_importances_
importance_dt = np.argmax(importance_list)+1
plt.plot(range(1,importance_list.shape[0]+1),importance_list)
plt.title("importance of feature")
plt.show()

The most important feature is the feature (question) 10, it matches my expect.

In [ ]:
text = "is same as"
if importance_dt != importance_pca:
    text = "is not same as"
print("The result {} the result of the PCA analysis. I hope they are same, because that means the model calculate the result base on the most important feature.".format(text))

__(m)__ Build a decision forest. Play around with the number of trees in the forest. Decide on a forest.

In [ ]:
#find best tree num
from sklearn.ensemble import RandomForestClassifier
tpr_list = []
tree_num_list = []
max_tree_num = 151
tree_num_step = 5
tree_num_min_step = 1
tree_num_range = [1,max_tree_num]
while tree_num_step>=tree_num_min_step:
    t_l = []
    t_n_l = []
    for tree_num in range(tree_num_range[0],tree_num_range[1]+1,tree_num_step):
        model_rf = RandomForestClassifier(n_estimators=tree_num,n_jobs=-1)
        model_rf.fit(training_set,training_label)
        validation_output = model_rf.predict(validation_set)
        validation_result = np.where(validation_label==validation_output,1,0)
        tpr = validation_result.sum()/validation_result.shape[0]
        t_l.append(tpr)
        t_n_l.append(tree_num)
    tpr_list.append(t_l)
    tree_num_list.append(t_n_l)
    max_tpr_tree_num = t_n_l[np.argmax(t_l)]
    print("range:[{},{}] step:{} tree_num:{} TPR:{:.2%}".format(tree_num_range[0],tree_num_range[1],tree_num_step,max_tpr_tree_num,np.max(t_l)))
    tree_num_range = [max_tpr_tree_num-tree_num_step*2,max_tpr_tree_num+tree_num_step*2]
    tree_num_step = int(tree_num_step/2)
best_tree_num = tree_num_list[-1][np.argmax(tpr_list[-1])]
tpr_rf = np.max(tpr_list[-1])
print("[Random Forest Classifier] best_tree_num:{} TPR:{:.2%}".format(best_tree_num,tpr_rf))
for i in range(len(tpr_list)):
    plt.plot(tree_num_list[i],tpr_list[i])
plt.title("TPR of tree number")
plt.show()

In [ ]:
#find best depth
max_depth = 18
start_depth = 1
tpr_list = []
for i in range(start_depth,max_depth+1):
    model_rf = RandomForestClassifier(n_estimators=best_tree_num,max_depth=i,n_jobs=-1)
    model_rf.fit(training_set,training_label)
    validation_output = model_rf.predict(validation_set)
    validation_result = np.where(validation_label==validation_output,1,0)
    tpr = validation_result.sum()/validation_result.shape[0]
    tpr_list.append(tpr)
best_depth_rf = np.argmax(tpr_list)+start_depth
tpr_rf = tpr_list[np.argmax(tpr_list)]
print("[Random Forest Classifier] best_tree_num:{} best_depth:{} TPR:{:.2%}".format(best_tree_num,best_depth_rf,tpr_rf))
plt.plot(range(start_depth,max_depth+1),tpr_list)
plt.title("TPR of depth")
plt.show()

__(n)__ Extract the most important features. Comment and compare with previously obtained results.

In [ ]:
model_rf = RandomForestClassifier(n_estimators=best_tree_num,max_depth=best_depth_rf,n_jobs=-1)
model_rf.fit(training_set,training_label)
importance_list = model_rf.feature_importances_
importance_rf = np.argmax(importance_list)+1
plt.plot(range(1,importance_list.shape[0]+1),importance_list)
plt.title("importance of feature")
plt.show()

The most important feature is the feature (question) 10, it is the same result as other models.

__(o)__ Compare the results of the in terms of confusion matrices, accuracy, precision, recall, and f-score. How well can we predict the partisan affiliation of a candidate based on their answers to a test? How does this compare with your intuition?

In [ ]:
model_nb = CategoricalNB()
model_nb.fit(training_set,training_label)
model_knn = knn(n_neighbors=best_k,n_jobs=-1)
model_knn.fit(training_set,training_label)
model_dt = DecisionTreeClassifier(ccp_alpha=best_ccp_alpha)
model_dt.fit(training_set,training_label)
model_rf = RandomForestClassifier(n_estimators=best_tree_num,max_depth=best_depth_rf,n_jobs=-1)
model_rf.fit(training_set,training_label)
model_list = [model_nb,model_knn,model_dt,model_rf]
model_name = ["Naive Bayes Classifier","KNN Classifier","Decision Tree Classifier","Random Forest Classifier"]

In [ ]:
result_list = []
for model in model_list:
    validation_output = model.predict(validation_set)
    confusion_matrix = np.zeros([len(label_dict),len(label_dict)])
    for l_i,label in enumerate(label_dict.keys()):
        tp = np.sum(np.where((validation_label==label) & (validation_output==label),1,0))
        for l_fp_i,label_fp in enumerate(label_dict.keys()):
            if label!= label_fp:
                fp = np.sum(np.where((validation_label==label_fp) & (validation_output==label),1,0))
                confusion_matrix[l_i,l_fp_i] = fp
            else:
                confusion_matrix[l_i,l_fp_i] = tp
    aprf_list = []
    tp_model = 0
    fp_model = 0
    fn_model = 0
    tn_model = 0
    for l_i,label in enumerate(label_dict.keys()):
        tp = confusion_matrix[l_i,l_i]
        fp = np.sum(confusion_matrix[l_i,:]) - tp
        fn = np.sum(confusion_matrix[:,l_i]) - tp
        tn = np.sum(confusion_matrix) - tp - fp - fn
        tp_model += tp
        fp_model += fp
        fn_model += fn
        tn_model += tn
        accuracy = (tp+tn)/(tp+fp+fn+tn)
        precision = tp / (tp+fp)
        recall =tp / (tp+fn)
        f_score = 2*precision * recall / (precision + recall)
        aprf_list.append([accuracy,precision,recall,f_score])
    accuracy_model = (tp_model+tn_model)/(tp_model+fp_model+fn_model+tn_model)
    precision_model = tp_model / (tp_model+fp_model)
    recall_model =tp_model / (tp_model+fn_model)
    f_score_model = 2*precision_model * recall_model / (precision_model + recall_model)
    result_list.append([confusion_matrix,accuracy_model,precision_model,recall_model,f_score_model,aprf_list])

In [ ]:
for r_i,[confusion_matrix,accuracy_model,precision_model,recall_model,f_score_model,aprf_list] in enumerate(result_list):
    print("{}\n[model] accuracy:{:.2%} precision:{:.2%} recall:{:.2%} f_score:{}".format(model_name[r_i],accuracy_model,precision_model,recall_model,f_score_model))
    for i,[accuracy,precision,recall,f_score] in enumerate(aprf_list):
        label = list(label_dict.keys())[i]
        print("{} accuracy:{:.2%} precision:{:.2%} recall:{:.2%} f_score:{}".format(label,accuracy,precision,recall,f_score))
    print("")

The f-score of models are nearly 0.8, so we can predict the partisan affiliation of most candidates(80%) based on their answers to a test.